# Installing Requirements (Run before going to Required Imports)

In [ ]:
!pip install energyflow           #for loading quark-gluon data
!pip install torch
!pip install torch-geometric
!pip install torch_cluster        #this might take 5-6 mins to install. Ateast that was the case on my device
!pip install scikit-learn


# Required Imports

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool, global_max_pool, GATConv, EdgeConv
from sklearn.model_selection import train_test_split
from torch_cluster import knn_graph
import energyflow

# Graph Based Quark-Gluon Classification Architectures

The objective of task-2 is to explore two graph based architectures for classification of quarks and gluon jets in the dataset provided in the task.

I have chosen the following graph based architectures:


1.   Graph Attention Networks (**GAT**)
2.   Dynamic Graph Convolutional Neural Networks (**DGCNN**)


**Before I explore these models, I would like to explain how I projected this dataset to a set of interconnected nodes and edges-**

Each jet in the dataset has been considered as a graph (i.e set of interconnected nodes and edges). There are 100,000 jets in total. Thus we would have 100,000 graphs.

Every jet is a 2-D array in the dataset. It is an array of particles. Every particle in the jet is a 1-D array consisting of 4 features (pt, rapidity, azimuthal angle, and pdgid)

Thus I am considering every particle in the jet as a node inside the graph. The features are basically attributes of that node. Since features can vary over several orders of magnitude, I also normalize each feature which helps in stabilizing training and ensures that all features contribute comparably when computing distances. The dataset is also padded with zeros as particles for jets that have lesser than the maximum number of particles. However, these don't provide any information thus they are removed.

Now coming to edges, I constructed edges for each node by using the k-Nearest Neighbors approach. For every node, I connected it to its k-NN based on euclidean distance in the feature space.  In the current case, using k=16 neighbors typically provides sufficient connectivity for the GAT or DGCNN to learn meaningful representations without overwhelming the model with too many edges.

Also, in the k-NN graph, I have not included self-loops (edges from a node to itself) since they do not provide additional relational information.

Additionaly, GAT uses a fixed (static) graph using the raw features. In case of DGCNN, the benefit is in dynamically recomputing the k-NN graph at each layer to capture evolving relationships as the node features get updated.

Once the nodes are processed by the network layers, I have applied global pooling to aggregate node-level information into a single, fixed-size graph-level representation. This aggregated vector is used for the final classification task.







The **create_graph_list** function below shows what I have written above.

It loads the data set as two arrays-

A 3-D array (X) of shape (N,M,d) where N is the number of jets, M is max number of particles per jet and d is the number of features per particle.

A 1-D array (Y) of the output labels (Quark/Gluon) for every jet

Then performs normalization, removing padding, k-NN graph finding.
It returns a list of PyG objects which are basically jets wrapped up with their respective labels. Run the cell below to preprocess the dataset and get it into appropriate for running through the models

In [3]:
def create_graph_list(k):

    X,Y=energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')     #loading the QG dataset X is 3-D array and Y is 1-D

    data_list=[]

    for i in range(X.shape[0]):                                      #Iterating through the jets to create a graph for each jet
        x=torch.tensor(X[i], dtype=torch.float)                      # shape: (M, d)
        x=(x-x.mean(dim=0))/x.std(dim=0)                             #Normalizing the features across particles (for each feature column).
        mask=(x.abs().sum(dim=1)>1e-8)                               #Removing padded particles (all input features 0)
        x=x[mask]
        edge_index=knn_graph(x,k=k,loop=False)                       #Finds k-NN for every node/particle in the graph/jet in feature space (uses Euclidean distance)
        label=torch.tensor([Y[i]],dtype=torch.long)                  #I am wrapping the label as a tensor as well to add to the PyG data object
        data_list.append(Data(x=x,edge_index=edge_index,y=label))    #creating a PyG data object and appending it to a list of graphs. Each graph/jet is a PyG object

    return data_list

**GAT:**

GAT layers learn to weigh each neighbor’s influence differently by computing attention scores. Multiple attention heads allow the model to capture different types of interactions. This is why I have chose GAT as one of my architectures as not all particles contribute equally to identifying a jet as quark- or gluon-initiated. GAT’s attention mechanism lets the network focus more on the most informative particle interactions. The heads enable the network to learn various aspects of the relationships between particles, which is crucial given the complex structure of jets.


After processing through three GAT layers, I have used global mean pooling to aggregate node features into a single graph-level feature, which is then fed into a fully connected layer for classification.

Below is the implementation of the GAT. Please run the cell so that it can be used later.

In [5]:
class QG_GAT(nn.Module):

  def __init__(self, in_channels, hidden_channels, out_channels=2, heads=4):

    super().__init__()
    #3 processing layers
    self.layer_1=GATConv(in_channels, hidden_channels, heads=heads, concat=True)             #'heads' attention heads and their outputs concatenated. Outputs are feature vectors of hidden channels
    self.layer_2=GATConv(hidden_channels*heads, hidden_channels, heads=heads, concat=True)   #Thus input channels is hidden_channels*heads. Same heads, concatenation and output feature vector size as as before
    self.layer_3=GATConv(hidden_channels*heads, hidden_channels, heads=1, concat=False)      #1 attention head only now therefore the fc layer gets a feature vector of hidden_channels size

    self.fc=nn.Linear(hidden_channels, out_channels)                                         #maps to final outputs

  def forward(self, x, edge_index, batch):

    x=functional.elu(self.layer_1(x, edge_index))                                            #Applying GAT layers with ELU activation.
    x=functional.elu(self.layer_2(x, edge_index))
    x=functional.elu(self.layer_3(x, edge_index))

    x=global_mean_pool(x, batch)

    return self.fc(x)

**DGCNN-**

Instead of using a fixed graph, DGCNN recomputes the graph (using k-NN) at every layer. This means that after each layer, as the node features change, the graph’s connectivity is updated accordingly. Thus, I used DGCNN the network learns, the optimal relationships between particles can change. DGCNN’s dynamic graph construction allows the network to update these relationships at every layer, capturing higher-level, context-dependent interactions.

DGCNN uses EdgeConv layers that consider both a node and its neighbors. EdgeConv layers are particularly good at learning local structures, which is important because the spatial and energy distributions of particles in a jet are key to distinguishing between quark and gluon jets.

After several EdgeConv layers, global pooling (both max and mean) aggregates the node features into a graph-level feature vector, which is then passed through a fully connected layer to yield class scores.


Below is the implementation of the DGCNN. Please run the cell so that it can be used later.

In [7]:
class QG_DGCNN(nn.Module):

  def __init__(self, in_channels, hidden_channels, out_channels=2, k=16):

    super().__init__()
    self.k=k
    #4 processing layers
    self.conv_layer_1=EdgeConv(nn.Sequential(nn.Linear(2*in_channels, hidden_channels),nn.ReLU(), nn.Linear(hidden_channels,hidden_channels)))            #Input dimension is 2*in_channels because features of a node and its neighbor are concatenated. Outputs a feature vector of size hidden channels
    self.conv_layer_2=EdgeConv(nn.Sequential(nn.Linear(2*hidden_channels, hidden_channels), nn.ReLU(), nn.Linear(hidden_channels,hidden_channels)))       #Input dimension is 2*hidden_channels (from previous layer's output)
    self.conv_layer_3=EdgeConv(nn.Sequential(nn.Linear(2*hidden_channels, hidden_channels), nn.ReLU(), nn.Linear(hidden_channels,hidden_channels)))
    self.conv_layer_4=EdgeConv(nn.Sequential(nn.Linear(2*hidden_channels, hidden_channels), nn.ReLU(), nn.Linear(hidden_channels,hidden_channels)))

    self.conv_layer_list=[self.conv_layer_1, self.conv_layer_2, self.conv_layer_3, self.conv_layer_4]                                #Save all EdgeConv layers in a list for iteration in the forward pass

    self.fc=nn.Linear(hidden_channels*2, out_channels)                                                                               #Takes the concatenated output from global max and mean pooling (2*hidden_channels) and maps it to out_channels classes


  def forward(self, x, edge_index, batch):

    for layer in self.conv_layer_list:
      edge_index = knn_graph(x, k=self.k, batch=batch, loop=False)                 #Dynamically computing the k-NN graph based on the current node features
      x=layer(x, edge_index)                                                       #Updating the node features using the current EdgeConv layer

    x_max=global_max_pool(x, batch)
    x_mean=global_mean_pool(x, batch)
    x=torch.cat([x_max, x_mean], dim=1)                                            #Concatenating both pooled representations to form a graph-level feature vector

    return self.fc(x)

Below are functions for training and testing. These are used for a single epoch and are iteratively run later in main() for multiple (20) epochs

Please run the cell so that they can be used later

In [8]:
def train_epoch(model, loader, optimizer, device):
    
    model.train()                                                 #Putting model in train mode
    total_loss=0
    for batch in loader:                                          #going through the batches here
        batch=batch.to(device)
        optimizer.zero_grad()                                     #I am clearing all previous gradient computations here
        forward=model(batch.x, batch.edge_index, batch.batch)     #Running the forward method in the given model here
        loss=functional.cross_entropy(forward, batch.y)           #Calculating cross entropy loss
        loss.backward()                                           #Backpropagation
        optimizer.step()                                          #Updating the parameters
        total_loss+=loss.item()*batch.num_graphs                  #Loss for each batch is weighted with the number of graphs in each batch
    return total_loss/len(loader.dataset)                         #Returns the average loss per graph/jet for an epoch

def test_epoch(model, loader, device):

    model.eval()                                                  #Putting model in evaluation mode
    correct=0
    for batch in loader:
        batch=batch.to(device)
        with torch.no_grad():                                     #Disabling gradient computation for evaluation
            forward=model(batch.x, batch.edge_index, batch.batch) #Running the forward method in the model here
            prediction=forward.argmax(dim=1)                      #Predicted class
            correct+=prediction.eq(batch.y).sum().item()          #Number of correctly predicted graphs
    return correct/len(loader.dataset)                            #Returning accuracy

Below is the main functions that trains and tests both these models on 'num_epochs' epochs. I am using 20 as num_epochs when I call main.

Please run the cell to train, test and generate best accuracy values when predicting with both models.

I only split the dataset into training and test sets for simplicity, but a validation set can be put in as well with a different split.

In [9]:
def main(num_epochs):

    device=torch.device('cpu' if torch.backends.mps.is_available() else 'mps')          #Checking if MPS (as I am training on a MacBook Pro) is available and using it if it is. Otherwise using CPU
    print(device) #Can be used for checking what device being used for training
    data_list=create_graph_list(k=16)                                                   #I am using k as 16 for k-NN
    train_data,test_data=train_test_split(data_list, test_size=0.2, random_state=42)    #I have used a 80:20 split for training and testing data

    train_loader=DataLoader(train_data, batch_size=32, shuffle=True)                    #I am doing batch processing here
    test_loader=DataLoader(test_data, batch_size=32, shuffle=False)                     #Each Batch contains 32 graphs/jets and training data batches have been shuffled

    in_channels=data_list[0].x.shape[1]                                                 #Number of input freatures per particle/node (i.e. d in (N,M,d))

    gat_model=QG_GAT(in_channels, hidden_channels=64, out_channels=2, heads=4).to(device)   #Instance of our Quark Gluon GAT Classifier
    dgcnn_model=QG_DGCNN(in_channels, hidden_channels=64, out_channels=2, k=16).to(device)  #Instance of the DGCNN Classifier

    gat_optimizer=torch.optim.Adam(gat_model.parameters(), lr=1e-3)                            #I am using ADAM for training with 10^-3 learning rate
    dgcnn_optimizer=torch.optim.Adam(dgcnn_model.parameters(), lr=1e-3)

    gat_best_accuracy, dgcnn_best_accuracy=0,0                                         #Starting Training and Testing here over num_epochs

    for epoch in range(num_epochs):

        gat_train_loss = train_epoch(gat_model, train_loader, gat_optimizer, device)
        gat_train_accuracy = test_epoch(gat_model, train_loader, device)
        gat_test_accuracy = test_epoch(gat_model, test_loader, device)

        dgcnn_train_loss = train_epoch(dgcnn_model, train_loader, dgcnn_optimizer, device)
        dgcnn_train_accuracy = test_epoch(dgcnn_model, train_loader, device)
        dgcnn_test_accuracy = test_epoch(dgcnn_model, test_loader, device)

        if gat_test_accuracy>gat_best_accuracy:
            gat_best_accuracy=gat_test_accuracy
            torch.save(gat_model.state_dict(), 'best_gat_model.pt')                    #Saving the best model parameters for later use. I can load into the model if I wish to

        if dgcnn_test_accuracy>dgcnn_best_accuracy:
            dgcnn_best_accuracy=dgcnn_test_accuracy
            torch.save(dgcnn_model.state_dict(), 'best_dgcnn_model.pt')

        print(f"Epoch: {epoch+1:02d}")
        print(f"GAT Loss: {gat_train_loss:.4f}, GAT Train Accuaracy: {gat_train_accuracy:.4f}, GAT Test Accuracy: {gat_test_accuracy:.4f}")
        print(f"DGCNN Loss: {dgcnn_train_loss:.4f}, DGCNN Train Accuaracy: {dgcnn_train_accuracy:.4f}, DGCNN Test Accuracy: {dgcnn_test_accuracy:.4f}")
        print()

    print(f"\nTraining complete")
    print("Best GAT Accuracy:", gat_best_accuracy)
    print("Best DGCNN Accuracy:", dgcnn_best_accuracy)

if __name__=="__main__":
    main(num_epochs=20)

cpu


KeyboardInterrupt: 